In [2]:
import os
import caffe
import pandas as pd
import numpy as np

import csv
import  sys
import  time

import warnings
warnings.filterwarnings('ignore')


In [3]:
def getExtension(name):
   fileName, fileExtension = os.path.splitext(name)
   return fileExtension
 

def isExtensionPermited(extension):
    extensions = ['jpeg', 'jpg', 'jpe']
    for x in extensions:
        if extension[:1] == '.':
            if extension[1:].lower() == x:
                return True
            elif extension.lower() == x:
                return True
    return False


nameFile = []
def lookupDirectory(path):
   if os.path.isdir(path):
       files = os.listdir(path)
       for i in files:
           if i + '/' != 'windows/':
               if os.path.isdir(path + i + '/'):
                   lookupDirectory(path + i + '/')
               if isExtensionPermited(getExtension(i)) == True:
                   #print i
                   nameFile.append(path+i)


In [4]:
diretorio = 'tier-16-images/'

lookupDirectory(diretorio)

In [7]:
size = (256,256)

start = time.time()

deploy_file = 'deploy.prototxt'
caffemodel_file = 'snapshot_iter_74880.caffemodel'
net = caffe.Classifier(deploy_file, caffemodel_file)

labels = pd.read_csv(os.path.join('vgg19_finetuned_all','labels.txt'), names=['label'])


c = csv.writer(open("fileEmotion_test.csv", "wb"))

for i in range(len(nameFile)):
    try:
        image = caffe.io.resize_image(caffe.io.load_image(nameFile[i]), size)
        image = [(image * 255).astype(np.float)]
        caffe_prob = net.predict(image, oversample=False)
        caffe_prediction = caffe_prob.argmax(axis=1)

        caffe_prediction_labels = [ labels[labels.index == x][['label']].values[0][0] for x in caffe_prediction ]
       
        print 'Arquivo: ',nameFile[i], ' cplTmp: ',caffe_prediction,' - ',caffe_prediction_labels
        c.writerow([nameFile[i],caffe_prediction[0]])

    except:
        #print 'Arquivo: ',nameFile[i], ' cplTmp: [\'ERROR\']'
        sys.stdout.write('error, ')
        sys.stdout.flush()  
        
end = time.time()
sys.stdout.write(str(i) + ' ('+ str(round(end - start,3)) +') , ')
sys.stdout.flush()

#c.to_csv('fileEmotion_test.csv.gz', compression='gzip')


Arquivo:  /home/greice/projeto/tst/4446094482_b15f3d3de5_m.jpg  cplTmp:  [2]  -  ['positivo']
Arquivo:  /home/greice/projeto/tst/metallica_.jpg  cplTmp:  [1]  -  ['neutro']
Arquivo:  /home/greice/projeto/tst/test.jpg  cplTmp:  [0]  -  ['negativo']
2 (6.236) , 